## Ultilizando a biblioteca schedule para o agendamento para geração do arquivo solicitado: 
- Alternativa na importação do notebook para outra maquina que vá rodar a rotina mensal, sem a necessidade de configurar manualmente atravez do SO ou solução cloud.

**SOLUÇÕES TECNICAS ALTERNATIVAS, VULGO GAMBIARRA...**

In [ ]:
!pip install schedule

In [2]:
import schedule
import time
import pandas as pd
import pyspark.sql.functions as func

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
spark = SparkSession.builder.getOrCreate()


def AgendarRotinaMensal():
    df_amostra_ultimo = spark.read.csv(r'C:\Users\arthu\Desktop\EPD_202208-00.csv', sep = ',', inferSchema=True, header=True)
    df_amostra_anterior = spark.read.csv(r'C:\Users\arthu\Desktop\EPD_202207-00.csv', sep = ',', inferSchema=True, header=True)
    time.sleep(60)
    
    print("Esquema amostra de Agosto:\n"
          "Total de linhas: ", df_amostra_ultimo.count(),"\n" # Conferindo se todas as linhas foram importadas por completo
          "Total de colunas: ", len(df_amostra_ultimo.columns)) 
    df_amostra_ultimo.printSchema()

    print("Esquema amostra de Julho:\n"
          "Total de linhas: ", df_amostra_anterior.count(),"\n" # Conferindo se todas as linhas foram importadas por completo
          "Total de colunas: ", len(df_amostra_anterior.columns)) 
    df_amostra_anterior.printSchema()
    
    time.sleep(30)

    Rotina_mensal = df_amostra_anterior.union(df_amostra_ultimo).distinct()
    time.sleep(30)

    Rotina_mensal.toPandas().to_csv('Warehouse/Rotina_mensal.csv', index=False)
    Rotina_mensal_pd = pd.read_csv(r'Warehouse/Rotina_mensal.csv')
    time.sleep(30)

    print("Total de linhas repetidas e não adicionadas: ",
          df_amostra_ultimo.count()+df_amostra_anterior.count()-Rotina_mensal_pd.shape[0])

schedule.every(30).days.do(AgendarRotinaMensal)

while True:
    schedule.run_pending()
    time.sleep(6000)

Esquema amostra de Agosto:
Total de linhas:  588498 
Total de colunas:  13
root
 |-- YEAR_MONTH: integer (nullable = true)
 |-- REGIONAL_OFFICE_NAME: string (nullable = true)
 |-- ICB_NAME: string (nullable = true)
 |-- PCO_NAME: string (nullable = true)
 |-- PRACTICE_NAME: string (nullable = true)
 |-- CHEMICAL_SUBSTANCE_BNF_DESCR: string (nullable = true)
 |-- BNF_DESCRIPTION: string (nullable = true)
 |-- QUANTITY: double (nullable = true)
 |-- ITEMS: integer (nullable = true)
 |-- TOTAL_QUANTITY: double (nullable = true)
 |-- ADQUSAGE: double (nullable = true)
 |-- NIC: double (nullable = true)
 |-- ACTUAL_COST: double (nullable = true)

Esquema amostra de Julho:
Total de linhas:  585868 
Total de colunas:  13
root
 |-- YEAR_MONTH: integer (nullable = true)
 |-- REGIONAL_OFFICE_NAME: string (nullable = true)
 |-- ICB_NAME: string (nullable = true)
 |-- PCO_NAME: string (nullable = true)
 |-- PRACTICE_NAME: string (nullable = true)
 |-- CHEMICAL_SUBSTANCE_BNF_DESCR: string (nullable